In [7]:
import numpy as np
import pandas as pd

# Load positive tweets
pos_tweets = []
with open('twitter-datasets/train_pos_full.txt', encoding = 'utf-8') as f:
    for line in f:
        pos_tweets.append(line)
        
# Load negative tweets
neg_tweets = []
with open('twitter-datasets/train_neg_full.txt', encoding = 'utf-8') as f:
    for line in f:
        neg_tweets.append(line)

In [8]:
all_tweets = np.array(pos_tweets + neg_tweets)

In [9]:
y = np.concatenate((np.ones(len(pos_tweets)), np.zeros(len(neg_tweets))))

random_idxs = np.random.permutation(len(y))

all_tweets = all_tweets[random_idxs]

y = y[random_idxs]

N_train = int(0.8*len(y))

train, val = all_tweets[:N_train], all_tweets[N_train:]
y_train, y_val = y[:N_train], y[N_train:]

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer 
 
tfidf_vectorizer=TfidfVectorizer(use_idf=True, ngram_range = (1,2)) 
X_train = tfidf_vectorizer.fit_transform(train)
X_val = tfidf_vectorizer.transform(val)

In [11]:
first_vector_tfidfvectorizer=X_train[0] 
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)


,tfidf
poster 38,0.232860
38 x26,0.232860
modern warplanes,0.228211
warplanes poster,0.228211
x26 75,0.224605
...,...
from cos,0.000000
from cory,0.000000
from cortez,0.000000
from corriere,0.000000


In [ ]:
from sklearn import linear_model

clf =  linear_model.SGDClassifier(loss = 'log', max_iter=int(1e7), tol=1e-5, verbose = False)


clf.fit(X_train, y_train)
train_acc = (clf.predict(X_train) == y_train).mean()
val_acc = (clf.predict(X_val) == y_val).mean()
print('Training set accuracy: {:.2f}% / validation set: {:.2f}%'.format(100*train_acc, 100*val_acc))

In [12]:
from sklearn.svm import LinearSVC

clf = LinearSVC(random_state=0, tol=1e-9, loss = 'squared_hinge', dual = True, C = 0.03)
clf.fit(X_train, y_train)
train_acc = (clf.predict(X_train) == y_train).mean()
val_acc = (clf.predict(X_val) == y_val).mean()
print('Training set accuracy: {:.2f}% / validation set: {:.2f}%'.format(100*train_acc, 100*val_acc))

Training set accuracy: 86.11% / validation set: 84.35%


In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=1000, n_iter=7, random_state=42)

X_train_ = svd.fit_transform(X_train)
X_val_ = svd.transform(X_val)


clf = LinearSVC(random_state=0, tol=1e-9, loss = 'squared_hinge', dual = False, C = 0.1)
clf.fit(X_train_, y_train)
train_acc = (clf.predict(X_train_) == y_train).mean()
val_acc = (clf.predict(X_val_) == y_val).mean()
print('Training set accuracy: {:.2f}% / validation set: {:.2f}%'.format(100*train_acc, 100*val_acc))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=0)  
clf.fit(X_train, y_train)


predictions = clf.predict(X_val)

In [ ]:
train_acc = (clf.predict(X_train) == y_train).mean()
val_acc = (clf.predict(X_val) == y_val).mean()
print('Training set accuracy: {:.2f}% / validation set: {:.2f}%'.format(100*train_acc, 100*val_acc))

### Predictions

In [29]:
test_tweets = []
with open('twitter-datasets/test_data.txt', encoding = 'utf-8') as f:
    for line in f:
        sp = line.split(',')
        index = sp[0]
        test_tweets.append(','.join(sp[1:]))
        

        
tfidf_vectorizer=TfidfVectorizer(use_idf=True, ngram_range = (1,2)) 
X_train_final = tfidf_vectorizer.fit_transform(all_tweets)
X_test = tfidf_vectorizer.transform(test_tweets)


clf = LinearSVC(random_state=0, tol=1e-9, loss = 'squared_hinge', dual = True, C = 0.03)
clf.fit(X_train_final, y)
train_acc = (clf.predict(X_train_final) == y).mean()
print('Training set accuracy: {:.2f}%'.format(100*train_acc))

Training set accuracy: 86.27%


In [30]:
predictions = clf.predict(X_test)
df = pd.DataFrame((2*predictions-1).astype(int), columns = ['Prediction'], index = np.arange(1, len(predictions)+1))
df.index.names = ['Id']
df.to_csv('submission_tfidf.csv')

array([0., 1., 0., ..., 0., 1., 0.])